In [4]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


CommaSeparatedListOutputParser
- 콤마 기준으로 나눔 + csv파일로 저장할때 용이

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print(" AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "./data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

JSON output parser
- JSON 형식으로 구조화된 데이터 생성 및 API응답이나 데이터분석 용이

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# OpenAI 모델 설정
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


{
    "missions": [
        {
            "mission_name": "뉴호라이즌스",
            "goal": "명왕성 및 카이퍼 벨트 탐사",
            "agency": "NASA"
        },
        {
            "mission_name": "화성 2020",
            "goal": "화성 표면 샘플 수집 및 생명체 존재 여부 조사",
            "agency": "NASA"
        },
        {
            "mission_name": "창어 4호",
            "goal": "달 뒷면 탐사 및 샘플 수집",
            "agency": "중국 국가우주국 (CNSA)"
        }
    ]
}


PandasDataFrameOutputParser
- pandas DataFrame과 LLM을 연동하여 데이터를 쉽게 조회하고 분석할 수 있도록 돕는 출력 파서
- .csv 데이터프레임의 데이터를 분석하기 가능

In [6]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 웹에서 Titanic 데이터셋 불러오기
df = pd.read_csv('data/titanic.csv')

format_instructions = parser.get_format_instructions()

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)
df.head(2)

NameError: name 'parser' is not defined

In [ ]:

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are a helpful assistant that interacts whti a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    You task is to answer the user's query by generating a command in the following format:
    {format_instructions}\n{query}\n",
    
    User Query: {query}
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)


# 체인 생성
chain = prompt | model | parser

try:
    print("Name 컬럼 출력")
    df_query = "Show the Name Column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    print("첫번째 행 출력")
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)
except Exception as e:
    print(f"오류발생: {e}")

Name 컬럼 출력
오류발생: Error code: 401 - {'error': {'message': 'Incorrect API key provided: gsk_b4ab********************************************N6K3. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
